🚀 MULTI-START TABU SEARCH - 10 relances

Relance  1:  1180.88 →   361.68 (Δ:  819.19)
Relance  2:  1046.19 →   361.68 (Δ:  684.51)
Relance  3:   942.24 →   361.68 (Δ:  580.56)
Relance  4:   981.96 →   382.21 (Δ:  599.75)
Relance  5:  1017.00 →   352.32 (Δ:  664.68)
Relance  6:  1212.76 →   352.32 (Δ:  860.44)
Relance  7:   984.07 →   360.15 (Δ:  623.92)
Relance  8:  1044.88 →   406.24 (Δ:  638.65)
Relance  9:  1054.75 →   352.32 (Δ:  702.43)
Relance 10:  1106.74 →   361.68 (Δ:  745.06)

📊 RÉSULTATS FINAUX
Meilleure solution: 352.32
Coût moyen: 365.23 ± 15.96
Plage: 352.32 - 406.24
Temps: 1.76s

🏆 Solution optimale: 352.32
Tournée: 13 → 1 → 11 → 4 → 0 → 9 → 3 → 12 → 19 → 14 → 18 → 10 → 15 → 2 → 7 → 16 → 17 → 8 → 5 → 6 → 13
